In [ ]:
import pandas as pd
import numpy as np
import os

import pandas as pd
import numpy as np
from collections import Counter
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score
import math
from sklearn.preprocessing import MinMaxScaler
import scipy
from sklearn import metrics

import numpy as np
from sklearn.metrics import mean_absolute_error
from scipy.stats import pearsonr

import seaborn as sns
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

from scipy.stats import linregress
from scipy import stats
from sklearn.utils import resample

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import permutation_test

def perm_test(group1, group2, n_permutations=10000):
    observed_diff = np.mean(group1) - np.mean(group2)
    
    def statistic(x, y):
        return np.mean(x) - np.mean(y)
    
    result = permutation_test(
        data=(group1, group2),
        statistic=statistic,
        permutation_type='independent',
        vectorized=False,  # importante: no usar vectorized
        n_resamples=n_permutations,
        alternative='two-sided',
        random_state=42
    )
    
    return observed_diff, result.pvalue

def cliffs_delta(x, y):
    n_x = len(x)
    n_y = len(y)
    all_pairs = [(xi, yi) for xi in x for yi in y]
    n_greater = sum(1 for xi, yi in all_pairs if xi > yi)
    n_less = sum(1 for xi, yi in all_pairs if xi < yi)
    
    delta = (n_greater - n_less) / (n_x * n_y)
    return delta

# Función para quitar outliers con IQR
def remove_outliers_iqr(df, col, thres=1.5):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - thres * IQR
    upper = Q3 + thres * IQR
    return df[(df[col] >= lower) & (df[col] <= upper)]

def mean_directional_accuracy(y_true, y_pred):
    
    differences = np.array(y_pred) - np.array(y_true) 
    signs = np.sign(differences)
    mde = np.mean(signs)
    return mde

def mean_absolute_error(y_true, y_pred):

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    absolute_errors = np.abs(y_pred - y_true)
    mae = np.mean(absolute_errors)
    
    return mae

def coef_pval(coef_array_mean_, X_, y_, y_p):

    n = X_.shape[0]
    t = coef_tval(coef_array_mean_, X_, y_, y_p)
    p = 2 * (1 - scipy.stats.t.cdf(abs(t), n - 1))
    return p


def coef_tval(coef_array_mean_, X_, y_, y_p):
    
    '''
        coef_tval for OLS of statsmodels
    '''
    
    a = np.array(coef_array_mean_[0][0]/ coef_se(X_, y_, y_p)[0])
    b = np.array(coef_array_mean_[1::].flatten() / coef_se(X_, y_, y_p)[1:])
    return np.append(a, b)


def coef_se(X_, y_, y_p):
    
    '''
        coef_se for OLS of statsmodels
    '''
    n = X_.shape[0]
    
    X1 = np.hstack((np.ones((n, 1)), np.matrix(X_)))
    se_matrix = scipy.linalg.sqrtm(
        metrics.mean_squared_error(y_, y_p) *
        np.linalg.inv(X1.T * X1)
    )
    return np.diagonal(se_matrix)

def directional_accuracy(predicted_values, true_values):

    predicted_values = np.array(predicted_values)
    true_values = np.array(true_values)    

    difference_direction = np.sign(predicted_values - true_values)    
    correct_direction_count = np.sum(difference_direction == 1) + np.sum(difference_direction == -1)
    
    directional_accuracy_score = correct_direction_count / len(predicted_values)
    
    return directional_accuracy_score



from sklearn.ensemble import GradientBoostingRegressor
from statsmodels.api import GLM
from statsmodels.api import families


from sklearn.model_selection import GridSearchCV, KFold
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from scipy.stats import linregress
import numpy as np
import pandas as pd
import math
import warnings
from skopt import BayesSearchCV
from skopt.space import Real, Integer

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

def remove_outliers_iqr(df, col, iqr_k=1.5):
    s = pd.to_numeric(df[col], errors="coerce")
    q1 = s.quantile(0.25)
    q3 = s.quantile(0.75)
    iqr = q3 - q1
    low = q1 - iqr_k * iqr
    high = q3 + iqr_k * iqr
    return df.loc[(s >= low) & (s <= high)].copy()

def cliffs_delta(x, y):
    x = pd.to_numeric(pd.Series(x).dropna(), errors="coerce").dropna().values
    y = pd.to_numeric(pd.Series(y).dropna(), errors="coerce").dropna().values
    nx, ny = len(x), len(y)
    if nx == 0 or ny == 0:
        return np.nan
    diffs = x[:, None] - y[None, :]
    n_greater = np.sum(diffs > 0)
    n_less = np.sum(diffs < 0)
    return float((n_greater - n_less) / (nx * ny))

def permutation_pvalue(x, y, n_perm=10000, seed=42):
    """Two-sided permutation test for mean difference"""
    rng = np.random.default_rng(seed)
    x = np.array(x.dropna())
    y = np.array(y.dropna())
    if len(x) == 0 or len(y) == 0:
        return np.nan
    obs_diff = np.abs(np.mean(x) - np.mean(y))
    combined = np.concatenate([x, y])
    nx = len(x)
    count = 0
    for _ in range(n_perm):
        rng.shuffle(combined)
        x_perm, y_perm = combined[:nx], combined[nx:]
        diff = np.abs(np.mean(x_perm) - np.mean(y_perm))
        if diff >= obs_diff:
            count += 1
    return count / n_perm

def chunk_list(lst, n):
    """Split list into chunks of size n."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

def plot_gap_by_diagnosis_clean_wrap(
    df,
    gap_col,
    diagnosis_list,
    thresholds=(0.1, 0.5, 1.0, 1.5),
    per_line=3,
    title_fontsize=8,
    n_perm=10000
):
    # Build original data by group
    group_data_orig = {}
    for diag in diagnosis_list:
        sub = df[df["clinical_diagnosis"] == diag][[gap_col]].copy()
        sub["Origen"] = diag
        group_data_orig[diag] = sub

    # Unique pairs without repetition
    all_pairs = [(diagnosis_list[i], diagnosis_list[j])
                 for i in range(len(diagnosis_list))
                 for j in range(i+1, len(diagnosis_list))]

    cn_pairs = [(a, b) for (a, b) in all_pairs if a == "CN"]
    other_pairs = [p for p in all_pairs if p not in cn_pairs]

    results_table = []

    fig, axes = plt.subplots(2, 2, figsize=(7.0, 8.0), sharey=True)
    axes = axes.flatten()

    for ax, iqr_k in zip(axes, thresholds):
        group_data_clean = {
            diag: remove_outliers_iqr(group_data_orig[diag], gap_col, iqr_k)
            for diag in diagnosis_list
        }

        cn_lines = []
        for a, b in cn_pairs:
            x = group_data_clean[a][gap_col]
            y = group_data_clean[b][gap_col]
            dval = cliffs_delta(x, y)
            pval = permutation_pvalue(x, y, n_perm=n_perm)
            cn_lines.append(f"{a}-{b}: {dval:.3f}")
            results_table.append([iqr_k, a, b, dval, pval])

        other_lines = []
        for a, b in other_pairs:
            x = group_data_clean[a][gap_col]
            y = group_data_clean[b][gap_col]
            dval = cliffs_delta(x, y)
            pval = permutation_pvalue(x, y, n_perm=n_perm)
            other_lines.append(f"{a}-{b}: {dval:.3f}")
            results_table.append([iqr_k, a, b, dval, pval])

        cn_wrapped = [" | ".join(chunk) for chunk in chunk_list(cn_lines, per_line)]
        other_wrapped = [" | ".join(chunk) for chunk in chunk_list(other_lines, per_line)]

        title_parts = [f"IQR={iqr_k}"] + cn_wrapped + other_wrapped
        ax.set_title("\n".join(title_parts), fontsize=title_fontsize)

        combined = pd.concat([group_data_clean[d] for d in diagnosis_list], ignore_index=True)
        sns.boxplot(data=combined, x="Origen", y=gap_col, order=diagnosis_list, ax=ax)
        ax.axhline(0, color='gray', linestyle='--', linewidth=1)
        ax.set_xlabel("")
        ax.set_ylabel(gap_col)

    plt.tight_layout()
    results_df = pd.DataFrame(results_table, columns=["IQR_k", "Group1", "Group2", "CliffsDelta", "p_value"])
    return fig, axes, results_df


In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import r2_score
from sklearn.inspection import permutation_importance

from scipy.stats import linregress
import shap


def run_nested_hgb(
    df_data,
    selected_hgb,
    n_splits_outer = 10,
    n_splits_inner = 5,
    random_state = 42,
    scaler_range=(0.05, 0.95),
    param_grid=None,
    n_perm_repeats: int = 30,
    shap_check_additivity: bool = False,
    verbose: int = 2
):


    if param_grid is None:
        param_grid = {
            "model__max_iter": [300, 400, 500, 600],
            "model__max_depth": [3, 5, 7],
            "model__learning_rate": [0.01, 0.05, 0.1],
        }

    if isinstance(selected_hgb, str):
        selected_hgb = [selected_hgb]
    selected_hgb = list(selected_hgb)

    y = df_data["demo_age"]
    X_selected = df_data[selected_hgb]

    kf_outer = KFold(n_splits=n_splits_outer, shuffle=True, random_state=random_state)

    r2_scores = []
    y_true_all, y_pred_all = [], []

    results_labels_df = pd.DataFrame(columns=[
        'y_labels', 'y_pred', 'y_pred_M1', 'y_pred_M2', 'y_pred_M3',
        'GAP', 'GAP_corrected_M1', 'GAP_corrected_M2', 'GAP_corrected_M3', 'ID'
    ])

    shap_values_sum = pd.Series(0.0, index=X_selected.columns)
    perm_values_sum = pd.Series(0.0, index=X_selected.columns)

    best_params_by_fold = []

    for fold, (train_idx, test_idx) in enumerate(kf_outer.split(X_selected), start=1):
        print(f" Fold {fold} (Nested CV)")

        X_train, X_test = X_selected.iloc[train_idx], X_selected.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        pipeline = Pipeline([
            ("scaler", MinMaxScaler(feature_range=scaler_range)),
            ("model", HistGradientBoostingRegressor(random_state=random_state))
        ])

        grid_search = GridSearchCV(
            estimator=pipeline,
            param_grid=param_grid,
            scoring="r2",
            cv=n_splits_inner,
            n_jobs=-1,
            verbose=verbose
        )

        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_
        best_params_by_fold.append(grid_search.best_params_)

        y_pred = best_model.predict(X_test)
        r2 = r2_score(y_test, y_pred)
        r2_scores.append(r2)

        print(f" R² fold {fold}: {r2:.4f} (best params: {grid_search.best_params_})")

        y_true_all.extend(y_test.tolist())
        y_pred_all.extend(y_pred.tolist())

        gap_test = y_pred - y_test
        gap_train_all = best_model.predict(X_train) - y_train

        train_ids = X_train.index
        diag_train = df_data.loc[train_ids, 'clinical_diagnosis']
        cn_mask = (diag_train == 'CN').values  

        if cn_mask.sum() >= 2:
            slope, intercept, _, _, _ = linregress(y_train.values[cn_mask], gap_train_all.values[cn_mask])
        else:
            slope, intercept = 0.0, 0.0 

        corrected_gap_M1 = gap_test - (slope * y_test + intercept)
        y_pred_M1 = y_test + corrected_gap_M1

        y_train_pred = best_model.predict(X_train)
        if cn_mask.sum() >= 2:
            slope_pred, intercept_pred, _, _, _ = linregress(y_train.values[cn_mask], y_train_pred[cn_mask])
            if slope_pred == 0:
                slope_pred = 1.0
        else:
            slope_pred, intercept_pred = 1.0, 0.0

        y_pred_M2 = (y_pred - intercept_pred) / slope_pred
        corrected_gap_M2 = y_pred_M2 - y_test

        y_pred_M3 = (y_pred - intercept_pred) / slope_pred
        corrected_gap_M3 = y_pred_M3 - y_test

        explainer = shap.Explainer(best_model.named_steps['model'], X_train)
        shap_values = explainer(X_test, check_additivity=shap_check_additivity)
        shap_values_mean = np.abs(shap_values.values).mean(axis=0)
        shap_values_sum += pd.Series(shap_values_mean, index=X_selected.columns)

        perm_importance = permutation_importance(
            best_model, X_test, y_test,
            n_repeats=n_perm_repeats,
            random_state=random_state,
            n_jobs=-1
        )
        perm_values_sum += pd.Series(perm_importance.importances_mean, index=X_selected.columns)

        result = np.column_stack((
            y_test, y_pred, y_pred_M1, y_pred_M2, y_pred_M3,
            gap_test, corrected_gap_M1, corrected_gap_M2, corrected_gap_M3
        ))
        temp_df = pd.DataFrame(result, columns=[
            'y_labels', 'y_pred', 'y_pred_M1', 'y_pred_M2', 'y_pred_M3',
            'GAP', 'GAP_corrected_M1', 'GAP_corrected_M2', 'GAP_corrected_M3'
        ])
        temp_df['ID'] = X_test.index

        results_labels_df = pd.concat([results_labels_df, temp_df], ignore_index=True)

    r2_mean = float(np.mean(r2_scores))
    r2_std = float(np.std(r2_scores))
    r2_global = float(r2_score(y_true_all, y_pred_all))

    print("\n R² promedio:", r2_mean, r2_std)
    print(" R² global:", r2_global)

    shap_importance_avg = (shap_values_sum / kf_outer.get_n_splits()).sort_values(ascending=False)
    perm_importance_avg = (perm_values_sum / kf_outer.get_n_splits()).sort_values(ascending=False)

    results_labels_df = results_labels_df.set_index('ID').sort_index()
    df_final = pd.concat(
        [df_data[['demo_age', 'clinical_diagnosis'] + selected_hgb], results_labels_df],
        axis=1
    )

    out = {
        "selected_hgb": selected_hgb,
        "df_final": df_final,
        "results_labels_df": results_labels_df,
        "r2_scores": r2_scores,
        "r2_mean": r2_mean,
        "r2_std": r2_std,
        "r2_global": r2_global,
        "y_true_all": y_true_all,
        "y_pred_all": y_pred_all,
        "best_params_by_fold": best_params_by_fold,
        "shap_importance_avg": shap_importance_avg,
        "perm_importance_avg": perm_importance_avg,
    }
    return out


## Load Data

In [ ]:
df_data = pd.read_excel('../Data/augmented_data.xlsx')

In [ ]:
selected_hgb = ['concreteness',
       'granularity_extraction', 'psycholinguistic_objective',
       'pitch_analysis', 'sentiment_analysis', 'talking_intervals',
       'verbosity']

## Regression

In [ ]:
results = run_nested_hgb(df_data, selected_hgb, n_splits_outer = 10, n_splits_inner = 5)